In [1]:
from utils.binaries import *
from utils.plotting import *

16:36:43 (   +1.6s) [INFO   ] -- import logging
16:36:43 (  +195ms) [INFO   ] -- import numpy as np
16:36:43 (   +13ms) [INFO   ] -- import uncertainties
16:36:44 (  +423ms) [INFO   ] -- import pandas as pd
16:36:44 (    +2ms) [WARNING] -- pandas support to be dropped at some point!
16:36:44 (    +5ms) [INFO   ] -- import binaries.tools as tools
16:36:44 (  +414ms) [INFO   ] -- import matplotlib.pyplot as plt
16:36:45 (  +579ms) [INFO   ] -- import seaborn as so
16:36:45 (    +3ms) [INFO   ] -- import plotting.tools as plot
16:36:45 (   +13ms) [DEBUG  ] -- font size set to 9.5
16:36:45 (    +1ms) [DEBUG  ] -- label size set to 13.0
16:36:45 (    +0ms) [DEBUG  ] -- figure size set to [6.6, 3.3]
16:36:45 (    +1ms) [DEBUG  ] -- markersize set to 2.0
16:36:45 (    +1ms) [DEBUG  ] -- usetex set to False


In [2]:
def read_file(run, station):

    path = f"/cr/data01/filip/Data/muonAcquisitionII/data/{run}/muonAcqStore.out_{station}.out"
    all_hist = lambda x: [int(e.split(" ")[-1]) for e in x]
    coinc_hist = lambda x: [int(e.split(" ")[-4]) for e in x]  # ...
    peak_histo, rates = [], []

    with open(path, "r") as source:

        baseline = 0

        is_peak_histogram = False
        is_all_histogram = False
        for i, line in enumerate(source.readlines()):
            line = line.strip()

            if line.startswith("Offset"):
                baseline = int(line.split(" ")[-1])

            if line == "Peak":
                is_peak_histogram = True
                continue
            elif line == "Charge":
                is_peak_histogram = False

            if is_peak_histogram:
                peak_histo.append(line)
            if line == "======ssd:":
                is_all_histogram = True
                continue

            if is_all_histogram:
                rates.append(line)

        bin0 = int(rates[0].split(" ")[3])

    return bin0 - baseline, np.cumsum(all_hist(rates[1:])[::-1])[::-1] / 61

In [3]:
import os

data = '/cr/data01/filip/Data/muonAcquisitionII/data'
offsets = {}
int_rates = {}
timestamps = []

for run in os.listdir(data):
    files = os.listdir(f"{data}/{run}")
    if int(run) > 90: continue

    for file in files:
        if file.endswith('.out'):
            if file.startswith('muonAcqStore'):
                station = int(file.split('.')[1][-3:])
                
                offset, rate = read_file(run, station)

                try:
                    offsets[station].append(offset)
                    int_rates[station].append(rate)

                except KeyError:
                    offsets[station] = [offset]
                    int_rates[station] = [rate]
            else: 
                timestamps.append(int(file.split('.')[0]))

tools.pickle_save('/cr/data01/filip/Data/muonAcquisitionII/time_offset_rates.pkl', [timestamps, offsets, int_rates])